In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install gym_anytrading

In [ ]:
# Credit: The code to the below is from Phil Tabor
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
class DeepQNetwork(nn.Module):
    
    def __init__(self,lr,input_dims,fc1_dims,fc2_dims,n_actions,  n_conv):
        super(DeepQNetwork, self).__init__()
        self.input_dims = input_dims
        self.fc1_dims = fc1_dims
        self. n_conv =  n_conv 
        self.pad = 1
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
                
        self.firstSEQ = T.nn.Sequential(T.nn.Conv1d(in_channels=self.input_dims[0], out_channels=n_conv, kernel_size=5, padding = self.pad +1), 
                                        T.nn.ReLU(),
                                        T.nn.Conv1d(in_channels=n_conv, out_channels=n_conv*2, kernel_size=3, padding = self.pad),
                                    
                                        T.nn.ReLU(),
                                        
                                        T.nn.Dropout(p=0.2))#T.nn.BatchNorm1d(n_conv*2))
                                        
        self.lstm = T.nn.LSTM(input_size= self.input_dims[-1],hidden_size=self.fc1_dims)
        self.secondSEQ = T.nn.Sequential(
                                        #T.nn.Linear(self.fc1_dims, self.fc1_dims ),
                                        
                                        T.nn.ReLU(),
                                        T.nn.Dropout(p=0.2),
                                        T.nn.Linear(self.fc1_dims , self.n_actions))
                                         
        

        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss = nn.MSELoss()
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        #self.device = T.device('cpu')
        self.to(self.device)
        
        
    def forward(self, state):
        x =self.firstSEQ(state)
        x,_ =self.lstm(state)
        x = x[:, -1, :]
        actions = self.secondSEQ(x)
        
        
        
        return actions
        


class Agent():
    def __init__(self, gamma, epsilon, lr, input_dims, batch_size, n_actions,max_mem_size=100000, eps_end=0.01, eps_dec=5e-4,  n_neurones = 128, n_conv = 128):
        self.gamma = gamma
        self.epsilon = epsilon
        self.eps_min = eps_end
        self.eps_dec = eps_dec
        self.n_conv = n_conv
        self.input_dims = input_dims
        self.lr = lr
        self.action_space = [i for i in range(n_actions)]
        self.mem_size = max_mem_size
        self.batch_size = batch_size
        self.mem_cntr = 0
        print('init dqn')
        self.Q_eval = DeepQNetwork(self.lr, n_actions=n_actions, input_dims=input_dims,fc1_dims=n_neurones, fc2_dims=n_neurones, n_conv = self.n_conv ).float()
        self.state_memory = np.zeros((self.mem_size, *input_dims), dtype=np.float32)
        self.new_state_memory = np.zeros((self.mem_size, *input_dims), dtype=np.float32)
        self.action_memory = np.zeros(self.mem_size, dtype=np.int32)
        self.reward_memory = np.zeros(self.mem_size, dtype=np.float32)
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.bool)
    def store_transition(self, state, action, reward, state_, done):
        index = self.mem_cntr % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.action_memory[index] = action
        self.reward_memory[index] = reward
        self.terminal_memory[index] = done
        self.mem_cntr += 1
    def choose_action(self, observation):
        if np.random.random() > self.epsilon:
            state = T.tensor([observation]).to(self.Q_eval.device)
            
            #actions = self.Q_eval.forward(state.reshape(1, self.input_dims[0]).float())
            actions = self.Q_eval.forward(state.reshape(1, self.input_dims[0],self.input_dims[-1]).float())              
            action = T.argmax(actions).item()

        else:
            action = np.random.choice(self.action_space)
        return action
    def learn(self):
        if self.mem_cntr < self.batch_size:
            return
        self.Q_eval.optimizer.zero_grad()
        max_mem = min(self.mem_cntr, self.mem_size)
        batch = np.random.choice(max_mem, self.batch_size, replace=False)
        batch_index = np.arange(self.batch_size, dtype=np.int32)
        state_batch = T.tensor(self.state_memory[batch]).to(self.Q_eval.device)
        new_state_batch = T.tensor(self.new_state_memory[batch]).to(self.Q_eval.device)
        reward_batch = T.tensor(self.reward_memory[batch]).to(self.Q_eval.device)
        terminal_batch = T.tensor(self.terminal_memory[batch]).to(self.Q_eval.device)
        action_batch = self.action_memory[batch]
        q_eval = self.Q_eval.forward(state_batch.float())[batch_index, action_batch]
        if self.gamma >0: 
            q_next = self.Q_eval.forward(new_state_batch.float())
            q_next[terminal_batch] = 0.0
        
            q_target = reward_batch + self.gamma * T.max(q_next, dim=1)[0]
        else:
            q_target = reward_batch 
        loss = self.Q_eval.loss(q_target, q_eval).to(self.Q_eval.device)
        loss.backward()
        self.Q_eval.optimizer.step()
        self.epsilon = self.epsilon - self.eps_dec if self.epsilon > self.eps_min else self.eps_min



In [ ]:
"""def preprocess_btc(X, D):
    size = 100
    Y = []
    b = X[size - 1]
    for i,x in enumerate(X[size:]):
        #print(X[i:i+size+1][-1],x)
        m = np.mean(X[i:i+size+1])
        s = np.std(X[i:i+size+1])
        Y.append([x, (x-m)/s, (x - b)/s])
        b = x
    return np.array(Y)"""
        
#np.mean(np.array([btc[1:], btc_diff]), axis=1)    


In [ ]:
import pandas as pd
import gym
import gym_anytrading
import matplotlib.pyplot as plt


def plot_learning_curve(x, scores, epsilons, filename="", lines=None):
    fig=plt.figure()
    ax=fig.add_subplot(111, label="1")
    ax2=fig.add_subplot(111, label="2", frame_on=False)
    ax.plot(x, epsilons, color="C0")
    ax.set_xlabel("Training Steps", color="C0")
    ax.set_ylabel("Epsilon", color="C0")
    ax.tick_params(axis='x', colors="C0")
    ax.tick_params(axis='y', colors="C0")
    N = len(scores)
    running_avg = np.empty(N)
    for t in range(N):
        running_avg[t] = np.mean(scores[max(0, t-20):(t+1)])
    ax2.scatter(x, running_avg, color="C1")
    ax2.axes.get_xaxis().set_visible(False)
    ax2.yaxis.tick_right()
    ax2.set_ylabel('Score', color="C1")
    ax2.yaxis.set_label_position('right')
    ax2.tick_params(axis='y', colors="C1")
    if lines is not None:
        for line in lines:
            plt.axvline(x=line)
    #plt.savefig(filename)
    
    


url = 'https://drive.google.com/file/d/1VNcpTlUzb9ATlXSzZBBhxCQ4oshSBX6F/view?usp=sharing'
url2='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv('/kaggle/input/bitcoin-price-usd/main.csv')
print(df.columns)
def normalize(x,m,s):
    return (x-m)/s

btc = np.array(df['Close'])
btc_diff = np.diff(btc)
#btc_diff_norm = (btc_diff - btc_diff.mean())/btc_diff.std()


data = []
for i, x in enumerate(btc[1:]):
    data.append([x, btc_diff[i]])
data = np.array(data)

len(data)
btc_mean = data.mean(axis = 0)   
btc_std = data.std(axis = 0)

In [ ]:
seq_len = 5
train_len = 50
lst_env = []
for i in range(int(len(data)/train_len) -1 ):
    if i>0:        
        env = gym.make('stocks-v0',df = df[1:], frame_bound=(seq_len + (i-1)*train_len ,i*train_len), window_size=seq_len)  
        lst_env.append(env)
        
lst_env_emp = []
for i in range(int((len(data)-train_len)/20) -1 ):
       
        env = gym.make('stocks-v0',df = df[1:], frame_bound=(seq_len + i ,train_len + i), window_size=seq_len)  
        lst_env_emp.append(env)

In [ ]:
import random
agent = Agent(gamma=0.3, epsilon=1., batch_size=100, n_actions=2,eps_end=0.01, input_dims=[1,seq_len], eps_dec=2e-3, lr=0.01, n_neurones = 64, n_conv = 32)

lst_env_shuf = lst_env_emp





for j in range(10):
    rand_j = int(np.random.random()*len(lst_env_emp) - 2)
    env = lst_env_emp[(j+1)*100]
    e = j
    
    
    n_games = 100
    rand_noise = 10
    indice = 1
    PF=[]
    scores, eps_history,X, TOTAL = [], [], [], [] 
    scaler_std = np.array([env.observation_space.sample()[indice] for i in range(1000)]).std(axis = 0)
    #print(scaler_std, 'scaler_std', scaler_std.shape)
    scaler_mean = np.array([env.observation_space.sample()[indice] for i in range(1000)]).mean(axis = 0)
    #print(scaler_mean, 'scaler_mean', scaler_mean.shape)
    #scaler_std = btc_std
    #scaler_mean = btc_mean
    for i in range(n_games):
        
        score = 0
        done = False
        observation_ = env.reset()
        observation_ = np.flip(observation_, 0)
        observation = normalize(observation_, scaler_mean,scaler_mean).T[indice,:]
        
        count = 0
        Act = []
        while not done:
            count+=1
            #action = agent.choose_action(observation)
            
            action = env.action_space.sample()
            observation_, reward, done, info = env.step(action)
            observation_ = np.flip(observation_, 0)
           
            r = (action*observation_[0][1] - (1-action)*observation_[0][1])
            #reward = r
            Act.append(action)
            
            score += reward
            agent.store_transition(observation, action, r/100 ,normalize(observation_, scaler_mean,scaler_mean).T[indice,:], done)
            agent.learn()
            observation = normalize(observation_, scaler_mean,scaler_mean).T[indice,:]
            
        
        print(i, ' , score = ', score,  " epsilon = ", agent.epsilon, " mean action ", np.mean(Act), 'samplePrice = ', observation_[-1][0])
        scores.append(score)
        eps_history.append(agent.epsilon)
        X.append(i)
        
    plot_learning_curve(X, scores, eps_history,"file")
    plt.show()
    plt.figure(figsize=(8,4))
    plt.cla()
    env.render_all()
    plt.show()
        
   

In [ ]:
import random
#agent = Agent(gamma=0.0, epsilon=1., batch_size=100, n_actions=2,eps_end=0.0, input_dims=[1,seq_len], eps_dec=5e-4, lr=0.00001, n_neurones = 1024, n_conv = 512)


fees = 0.00

scores, eps_history,X, TOTAL = [], [], [], [] 
for j in range(10):
    rand_j = int(np.random.random()*len(lst_env_emp) - 2)
    env = lst_env_emp[j+60]
    e = j
   
   
    
    
    n_games = 1
    rand_noise = 10
    indice = 1
    PF=[]
    for i in range(n_games):
        
        score = 0
        done = False
        observation = normalize(env.reset(), btc_mean,btc_std).T[indice,:]
        observation_ = np.flip(observation_, 0)
        #observation =env.reset()
        count = 0
        Act = []
        while not done:
            count+=1
            action = agent.choose_action(observation)
            #action = env.action_space.sample()
            observation_, reward, done, info = env.step(action)
            observation_ = np.flip(observation_, 0)
            #print(observation, observation_)
            PF.append([observation_[0][0], action])
            r = (action*observation_[0][1] - (1-action)*observation_[0][1])
            reward = r
            Act.append(action)
            #print(r)
            score += reward
            #agent.store_transition(observation, action, reward ,normalize(observation_, btc_mean,btc_std).T[indice,:], done)
            #agent.learn()
            observation = normalize(observation_, btc_mean,btc_std).T[indice,:]
        
        scores.append(score)
        eps_history.append(agent.epsilon)
        X.append(e)
        print(e, ' , score = ', score,  " epsilon = ", agent.epsilon, " mean action ", np.mean(Act), 'samplePrice = ', observation_[0][0])
    plt.figure(figsize=(15,6))
    plt.cla()
    env.render_all()
    plt.show()

    def de_normalize(x, m, s):
        return np.array(x*s + m)
    total = score
    
    print('total = ', total)
        
    TOTAL.append([total, int(count_actions), total/count_actions])
#plot_learning_curve(X, scores, eps_history, "plot_learning_curve")

print(np.mean(TOTAL,axis=0))


In [ ]:
plot_learning_curve(X, scores, eps_history,"dddd")
observation_

In [ ]:


if __name__ == '__main__':
    
    env = lst_env_emp[0]
    n_games = 1
    rand_noise = 10
    indice = 1
    for i in range(n_games):
        
        score = 0
        done = False
        observation = env.reset()
        print(np.flip(observation,0))
        print(btc[1:21])
        
        #observation =env.reset()
        count = 0
        Act = []
        PF = []
        while not done:
            count+=1
            action =0
            
            observation_, reward, done, info = env.step(action)
            #print(observation, observation_)
            print(np.flip(observation_,0))
            break
            r = (-action*observation_[0][1] + (1-action)*observation_[0][1])
            PF.append([observation_[0][0], action])
            reward = r/10
            Act.append(action)
            
            score += reward
            agent.store_transition(observation, action, reward ,normalize(observation_, btc_mean,btc_std).T[indice,:], done)
            #agent.learn()
            observation = normalize(observation_, btc_mean,btc_std).T[indice,:]
        
        scores.append(score)
        eps_history.append(agent.epsilon)
        X.append(e)
        print(e, ' , score = ', score,  " epsilon = ", agent.epsilon, " mean action ", np.mean(Act))
        """plt.figure(figsize=(15,6))
        plt.cla()
        env.render_all()
        plt.show()
        """